In [3]:
import pandas as pd
from pandas import DataFrame
import math
import matplotlib.pyplot as plt
from matplotlib import path
import numpy as np
from copy import deepcopy
from aorcover import aorcover

#p = path.Path([(0,0), (0, 1), (1, 1), (1, 0)])  # square with legs length 1 and bottom left corner at the origin
#p.contains_points([(.5, .5)])

In [54]:
#accepts two sets of coordinates, each cooresponding to a polygon
#two polygons intersect if either one contains any of the other's vertices
def poly_intersect(coorarray1, coorarray2):
    poly1 = path.Path(coorarray1)
    poly2 = path.Path(coorarray2)
    p1p2 = poly1.contains_points(coorarray2)
    p2p1 = poly2.contains_points(coorarray1)
    return any(np.concatenate((p1p2, p2p1)))

In [55]:
#def mosaic_planner1(aordict):
#    mosaic_plan={}
#    for key1, value1 in aordict.iteritems():
#        overlaplist =[]
#        for key2, value2 in aordict.iteritems():
#            if (key1 != key2) & poly_intersect(value1, value2):
#                overlaplist.append(key2)
#        mosaic_plan[key1] = overlaplist
#    return mosaic_plan

In [56]:
#takes a dictionary of (aor, cornercoordinates) pairs 
#for each of the aor in the dictionary, finds all aors that have overlap with it
#put all of these aors in the same group
#return a list of these groups
def group_overlap_aor(adict):
    overlaplist=[]
    for key1, value1 in adict.iteritems():
        group = [key1]
        for key2, value2 in adict.iteritems():
            if (key1 != key2) & poly_intersect(value1, value2):
                group.append(key2)
        overlaplist.append(sorted(group))
    return sorted(overlaplist)         

In [57]:
#group_overlap_aor will contiain groups that are subgroups or duplicates
#this function removes groups that are subgroup of equal or larger groups to minimize mosaic workload
def remove_duplicate(ovl):
    ovl_copy = deepcopy(ovl)
    for i in range(0, len(ovl)):
        #for j in list(range(0,i))+list(range(i+1,len(ovl))):
        for j in list(range(i+1,len(ovl))):
            if set(ovl[i]) <= set(ovl[j]):
                #print ovl[i]
                #print 'is susbset of '
                #print ovl[j]
                #print '\n'
                try:
                    ovl_copy.remove(ovl[i])
                    print 'removed '+ ovl[i]+ '\n'
                except:
                    pass
            elif set(ovl[j]) <= set(ovl[i]):
                try:
                    ovl_copy.remove(ovl[j])
                    print 'removed '+ ovl[j]+ '\n'
                except:
                    pass
    return ovl_copy

In [58]:
#mosaic planner combines group_overlap_aor and remove_duplicate together
def mosaic_planner(adict):
    return remove_duplicate(group_overlap_aor(adict))

In [59]:
def write_plan(plan, path='/home/kecai/w49/mosaic_plan.txt'):
    target = open(path, 'w')
    i = 1
    for item in sorted(plan):
        target.write('Task '+ str(i)+': '+ str(item))
        target.write('\n')
        i = i+1
    target.close()
    return 0

In [60]:
aordf = pd.read_csv('/home/kecai/w49/aorcor.csv', sep = ',', header = 0)
aordict = {}
for column in aordf:
    aordict[aordf[column].name]=aordf[column].values.reshape(4,2)

In [61]:
hdr_hc = {}
hdr_hc ['4389888'] = aordict ['4389888']

In [62]:
non_hdr_hc = {}
non_hdr_hc['11018240'] = aordict['11018240']
non_hdr_hc['11017984'] = aordict['11017984']

In [63]:
group_overlap_aor(non_hdr_hc)

[['11017984', '11018240'], ['11017984', '11018240']]

In [64]:
#split aordict into two groups: HDR mode and non-HDR mode
hdr_mc={}
non_hdr_lc={}
for key, value in aordict.iteritems():
    if key.startswith('45'):
        hdr_mc[key] = value
    elif key.startswith('119') | key.startswith('7'):
        non_hdr_lc[key] = value

In [65]:
print 'hdr high coverage'
for key, value in hdr_hc.iteritems():
    print key
print '\n non hdr high coverage'
for key, value in non_hdr_hc.iteritems():
    print key
print '\n hdr medium coverage'
for key, value in hdr_mc.iteritems():
    print key
print '\n non hdr low coverage'
for key, value in non_hdr_lc.iteritems():
    print key

hdr high coverage
4389888

 non hdr high coverage
11018240
11017984

 hdr medium coverage
45949952
45928960
45909248
45923328
45940224
45934848
45906176
45914880
45917184
45945600

 non hdr low coverage
7283968
11963904
11972096
11968768
11966976
11971072
11972864
11973376


In [66]:
non_hdr_lc_plan = mosaic_planner(non_hdr_lc)

In [67]:
hdr_mc_plan = mosaic_planner(hdr_mc)

In [68]:
hdr_hc_plan = mosaic_planner(hdr_hc)

In [69]:
non_hdr_hc_plan = mosaic_planner(non_hdr_hc)

In [70]:
print non_hdr_hc_plan

[['11017984', '11018240']]


In [71]:
for item in hdr_mc_plan:
    print item

['45906176', '45923328', '45940224']
['45906176', '45934848', '45940224']
['45906176', '45934848', '45945600']
['45909248', '45914880', '45917184']
['45909248', '45914880', '45949952']
['45909248', '45917184', '45923328']
['45914880', '45928960', '45949952']
['45917184', '45923328', '45940224']


In [72]:
for item in non_hdr_lc_plan:
    print item

['11963904', '11966976', '11972096']
['11963904', '11968768', '11973376']
['11963904', '11972096', '11973376']
['11966976', '11971072', '11972096', '7283968']
['11968768', '11972864', '11973376']


In [22]:
i=1
for item in non_hdr_lc_plan:
    aorcover(aordict, \
             colored = item, \
             path = '/home/kecai/w49/non_hdr_lc_plan'+str(i))
    i=i+1

In [383]:
i=1
for item in hdr_mc_plan:
    aorcover(aordict, \
             colored = item, \
             path = '/home/kecai/w49/hdr_mc_plan'+str(i))
    i=i+1

In [23]:
write_plan(non_hdr_lc_plan, path='/home/kecai/w49/non_hdr_lc_plan.txt' )

0

In [24]:
write_plan(hdr_mc_plan, path='/home/kecai/w49/hdr_mc_plan.txt')

0

In [27]:
write_plan(hdr_hc_plan, path='/home/kecai/w49/hdr_hc_plan.txt')

0

In [73]:
write_plan(non_hdr_hc_plan, path='/home/kecai/w49/non_hdr_hc_plan.txt')

0